In [25]:
import numpy as np
import pandas as pd
# import openai
# from openai.embeddings_utils import get_embedding
from transformers import GPT2Tokenizer
from tqdm import tqdm, trange
import torch

In [2]:
path = 'data/fine-food-reviews/Reviews.csv'

In [15]:
df = pd.read_csv(path)
df = df[:50000]

In [4]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
sample = df.Text[0]
sample

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading: 100%|██████████| 1.04M/1.04M [00:00<00:00, 3.09MB/s]
c:\Users\Shaz\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shaz\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 3.

In [13]:
tokenizer(sample)

{'input_ids': [40, 423, 5839, 1811, 286, 262, 28476, 414, 32530, 3290, 2057, 3186, 290, 423, 1043, 606, 477, 284, 307, 286, 922, 3081, 13, 383, 1720, 3073, 517, 588, 257, 20798, 621, 257, 13686, 6174, 290, 340, 25760, 1365, 13, 2011, 45246, 318, 957, 17479, 290, 673, 5763, 689, 428, 1720, 1365, 621, 220, 749, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
sum = 0
for text in tqdm(df.Text):
    sum += len(text.split())

print(sum / len(df))

100%|██████████| 50000/50000 [00:00<00:00, 346682.54it/s]

79.38386


In [23]:
df.Score = df.Score - 1

In [27]:
df = df.drop(df.columns.difference(['Text', 'Score']), axis=1)

In [28]:
train_df = df.sample(frac=0.6)
val_df = df.drop(train_df.index).sample(frac=0.25)
test_df = df.drop(train_df.index).drop(val_df.index)

In [30]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)